# Quantum Game Theory - Sudoku

Sudoku is a numerical logic puzzle that involves filling a 9x9 grid divided into nine 3x3 subgrids or "boxes." Each row, column, and box must contain all the digits from 1 to 9 without repetition. The puzzle begins with some of the cells pre-filled with numbers, serving as clues. The objective is to complete the grid while adhering to the rule that each number can appear only once in each row, column, and box. The challenge and complexity of the puzzle vary based on the number and placement of the initial digits.

This is a new quantum algorithm designed specifically to tackle Sudoku puzzles. This development isn't just about gaming; it demonstrates the robust capabilities of the Dynex quantum computing platform. By harnessing the power of quantum mechanics, this algorithm not only speeds up solutions for recreational puzzles but also showcases potential applications in complex logistics, optimization problems, and beyond.

Coded By: Y3TI & Sam Rahmeh - v2

In [ ]:
import numpy as np
import dimod
import dynex
from qubovert import boolean_var
import matplotlib.pyplot as plt 

In [ ]:
# Define Sudoku Puzzle Size (4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256)

# Either True or False
MainNet = False
# Testnet maximum N = 4   |   # Mainnet can run all factors
N = 16

In [ ]:
def generateSudokuPlot(matrix, values=True):
    n = matrix.shape[0]
    # Set figure size proportional to the size of the Sudoku grid
    fig, ax = plt.subplots(figsize=(n * 0.6, n * 0.6))
    
    if values:
        ax.matshow(matrix[tuple([n - i for i in range(1, n + 1)]), :], cmap=plt.cm.Blues, alpha=0.4)
        min_val, max_val = 0, n
        for i in range(n):
            for j in range(n):
                ax.text(i, j, str(matrix[n - j - 1, i]), va='center', ha='center')
        ax.set_xlim(-0.5, n - 0.5)
        ax.set_ylim(-0.5, n - 0.5)
    else:
        ax.matshow(matrix, cmap=plt.cm.Blues, alpha=0.4)
    
    # Draw grid lines for mini subgrids
    sq_N = int(np.sqrt(n))
    for i in range(1, sq_N):
        ax.axhline(i * (n // sq_N) - 0.5, color='black', linewidth=2)
        ax.axvline(i * (n // sq_N) - 0.5, color='black', linewidth=2)
    
    plt.show()

def checkSudoku(grid):
    N = grid.shape[0]
    sq_N = int(np.sqrt(N))
    for i in range(N):
        if len(set(grid[i, :])) != N or len(set(grid[:, i])) != N:
            return False
    for i in range(0, N, sq_N):
        for j in range(0, N, sq_N):
            subgrid = grid[i:i+sq_N, j:j+sq_N]
            if len(set(subgrid.flatten())) != N:
                return False  
    return True

# Function to initialize the Sudoku puzzle
def solveSudoku(board, N):
    empty = findEmptyLocation(board, N)
    if not empty:
        return True
    row, col = empty
    for num in range(1, N+1):
        if isValid(board, row, col, num, N):
            board[row][col] = num
            if solveSudoku(board, N):
                return True
            board[row][col] = 0
    return False
    
def isValid(board, row, col, num, N):
    sq_N = int(np.sqrt(N))
    for x in range(N):
        if board[row][x] == num or board[x][col] == num:
            return False
    startRow = row - row % sq_N
    startCol = col - col % sq_N
    for i in range(sq_N):
        for j in range(sq_N):
            if board[i + startRow][j + startCol] == num:
                return False
    return True
    
def findEmptyLocation(board, N):
    for i in range(N):
        for j in range(N):
            if board[i][j] == 0:
                return (i, j)
    return None
    
def initializeSudoku(N):
    while True:
        num_bits = N**3
        sq_N = int(np.sqrt(N))
        z = np.array([i+1 for i in range(N)], dtype=np.int64)
        
        # Set initial values.
        idx_init = np.random.choice(np.arange(N*N), N, replace=False)
        a = np.array([[i for i in range(N)]+idx_init[j]*N for j in range(N)]).ravel()
        x_init = {a[i] : int(i%N == i//N) for i in range(N*N)}

        init_vec = np.zeros((num_bits, 1), dtype=np.int64)
        idx = np.array([k for (k, v) in x_init.items() if v])
        init_vec[idx] = 1

        sudoku_init = (np.kron(np.identity(N**2, dtype=np.int64), z) @ init_vec).reshape((N, N))
        
        # Check if the initial Sudoku is solvable
        board = sudoku_init.copy().astype(int)
        if solveSudoku(board, N):
            return sudoku_init, x_init

# Ensure N is a perfect square and at least 4
def isPerfectSquare(n):
    return int(np.sqrt(n))**2 == n

if not isPerfectSquare(N) or N < 4:
    raise ValueError("[DYNEX] N must be a perfect square number. (4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256)")

In [ ]:
# Initialize the Sudoku Puzzle
print("[DYNEX] GENERATING SOLVABLE SUDOKU PUZZLE")
sudoku_init, x_init = initializeSudoku(N)
generateSudokuPlot(sudoku_init)

In [ ]:
# Compute the Sudoku Puzzle
solution_found = False
attempts = 0

NumReads = 10000
Annealing = 1000

while not solution_found and attempts <= 3:  # Add a limit to avoid infinite loops
    num_bits = N**3
    sq_N = int(np.sqrt(N))
    z = np.array([i+1 for i in range(N)], dtype=np.int64)

    # Initializing identity matrices and constructing constraints for rows and columns with high penalties
    penaltyRC = 10000 
    penaltySubGrid = 8000 
    penalty_value = 10000 
    idN = np.identity(N)
    idSqN = np.identity(sq_N)

    aRC = np.concatenate((np.kron(np.kron(idN, np.ones((1, N))), idN),  
                          np.kron(np.kron(np.ones((1, N)), idN), idN)))  
    aSubGrid = np.kron(np.kron(np.kron(idSqN, np.ones((1, sq_N))), 
                               np.kron(idSqN, np.ones((1, sq_N)))), idN)
    bRC = np.ones((2 * N * N, 1))
    bSubGrid = np.ones((N * N, 1))

    QRC = penaltyRC * (aRC.T @ aRC - 2 * np.diag(np.diag(aRC.T @ aRC)))
    QSubGrid = penaltySubGrid * (aSubGrid.T @ aSubGrid - 2 * np.diag(np.diag(aSubGrid.T @ aSubGrid)))

    qVal = np.zeros((num_bits, num_bits))
    for i in range(num_bits):
        block_row = i // N 
        for j in range(block_row * N, (block_row + 1) * N): 
            if i != j:
                qVal[i, j] = penalty_value 
    Q = QRC + QSubGrid + qVal

    # Creating BQM from the QUBO matrices
    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
    for i in range(num_bits):
        for j in range(i, num_bits):
            if i == j:
                bqm.add_variable(i, Q[i, i])
            else:
                bqm.add_interaction(i, j, Q[i, j])
    bqm.offset += penaltyRC * (bRC.T @ bRC)[0, 0]
    bqm.offset += penaltySubGrid * (bSubGrid.T @ bSubGrid)[0, 0]

    for i, val in x_init.items():
        bqm.fix_variable(i, val)

    # Compute on Dynex
    model = dynex.BQM(bqm)
    print("[DYNEX] BULDING BQM & UPLOADING JOB TO SAMPLER")    
    sampler = dynex.DynexSampler(model, mainnet=MainNet, description='Quantum Sudoku Solution', bnb=False)
    sampleset = sampler.sample(num_reads=NumReads, annealing_time=Annealing, debugging=False, alpha=10, beta=1)
    solution = sampleset.first.sample
    if N <= 4: 
        print("\n", solution, "\n")
    
    # Convert solution into sudoku grid
    sol_vec = np.zeros((num_bits, 1), dtype=np.int64)
    for i in range(num_bits):
        if i in x_init:
            sol_vec[i] = x_init[i]
        else:
            sol_vec[i] = solution[i]
    sudoku_sol = (np.kron(np.identity(N**2, dtype=np.int64), z) @ sol_vec).reshape((N, N))

    if checkSudoku(sudoku_sol):
        solution_found = True
        print("[DYNEX] THE SOLUTION IS VALID\n")
        generateSudokuPlot(sudoku_sol)
    else:
        print("[DYNEX] THE SOLUTION IS INVALID, RETRYING QUBO")
        NumReads = NumReads + 500       # Auto Increase the Number of Reads
        Annealing = Annealing + 500     # Auto Increase the Annealing
        print("[DYNEX] INCREASING NUMREADS TO : ", NumReads)
        print("[DYNEX] INCREASING ANNEALING TO : ", Annealing)
    attempts += 1

if not solution_found:
    print("[DYNEX] THE SOLUTION NOT FOUND IN 3 ATTEMPTS. PLEASE INCREASE ANNEALING")